In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import json
import ast

In [2]:
def clean_yelp(df1,df2):
    """Takes in 2 dataframes from Yelp and cleans them, returning relevant columns.""" 
    """If df2 does not exist, pass 'df2 = None'."""
    #check if df2 exists and set final to either equal the concatted form of df1 and df2
    #or just df1 if df2 doesn't exist
    if df2 is not None:
        final = pd.concat([df1,df2])
    else:
        final = df1
    #drop duplicates by restaurant ID
    final = final.drop_duplicates(subset = "id",keep = "first").reset_index(drop = True)
    #get main restaurant category
    my_list = []
    for entry in range(len(final['categories'])):
        #make sure entry is not nan and can be read (must be str)
        if type(final['categories'][entry])== float:
            my_list.append(np.nan)
            continue
        #read in the entry and convert to dictionary
        x = ast.literal_eval(final['categories'][entry])
        #get first alias as this will be the main restaurant category
        x = x[0]['alias']
        my_list.append(x)
    final["Restaurant Category 1"] = pd.Series(my_list)
    #see whether pickup, delivery, or reservations are available
    pickup_list = []
    delivery_list = []
    reserve_list = []
    for entry in range(len(final['transactions'])):
        #make sure entry is not nan and can be read (must be str)
        if type(final['transactions'][entry])== float:
            continue
        y = ast.literal_eval(final['transactions'][entry])
        if "pickup" in y:
            pickup_list.append(1)
        else:
            pickup_list.append(0)
        if "delivery" in y:
            delivery_list.append(1)
        else:
            delivery_list.append(0)
        if "restaurant_reservation" in y:
            reserve_list.append(1)
        else:
            reserve_list.append(0)
    final['Pickup Available'] = pd.Series(pickup_list)
    final['Delivery Available'] = pd.Series(delivery_list)
    final["Reservation Available"] = pd.Series(reserve_list)
    #create price column
    my_prices = []
    for entry in range(len(final['price'])):
        #find number of dollar signs in price and assign value accordingly
        if final['price'][entry]=="$":
            my_prices.append("Low")
        elif final['price'][entry]=="$$":
            my_prices.append("Medium")
        elif final['price'][entry]=="$$$":
            my_prices.append("High")
        elif final['price'][entry]=="$$$$":
            my_prices.append("Very High")
        else:
            my_prices.append(np.nan)
    final['Price Level'] = pd.Series(my_prices)
    #find number of photos
    num_photos = []
    for entry in range(len(final['photos'])):
        if type(final['photos'][entry])!=str:
            num_photos.append(np.nan)
            continue
        x = ast.literal_eval(final['photos'][entry])
        #use length of x to determine the number of photos
        x = len(x)
        num_photos.append(x)
    final['number of photos'] = pd.Series(num_photos)
    #find city and state of restaurant
    city = []
    state = []
    for entry in range(len(final['location'])):
        if type(final['location'][entry])!=str:
            city.append(np.nan)
            state.append(np.nan)
            continue
        #create dictionary based on entry and find the city and state
        x= ast.literal_eval(final['location'][entry])
        y = x['city']
        z = x['state']
        city.append(y)
        state.append(z)
    final['City'] = pd.Series(city)
    final['State'] = pd.Series(state)
    #find opening (start) and closing (end) times of restaurants
    start_time = []
    end_time = []
    for entry in range(len(final['hours'])):   
        if type(final['hours'][entry]) == float:
            start_time.append(np.nan)
            end_time.append(np.nan)
        else:
            x = ast.literal_eval(final['hours'][entry])
            start_time.append(x[0]['open'][0]['start'])
            end_time.append(x[0]['open'][0]['end'])
    final['Opening Time'] = pd.Series(start_time)
    final['Closing Time'] = pd.Series(end_time)
    
    #find whether messaging is available from restaurant
    messaging = []
    for entry in range(len(final['messaging'])):      
        if type(final['messaging'][entry])!=float:
            messaging.append(1)
        else:
            messaging.append(0)
    final["Has Messaging"] = pd.Series(messaging)
    #keep certain columns
    keep = ['id','is_closed',
    'is_claimed',
    'review_count',
    'rating',
    'Restaurant Category',
    'Pickup Available',
    'Delivery Available',
    "Reservation Available",
    'Price Level',
    'number of photos',
    'City',
    'State',
    'Opening Time',
    'Closing Time',
    'Has Messaging']
    final = final[final.columns[final.columns.isin(keep)]]
    return final

In [3]:
dallas = pd.read_csv("All_Available_Dallas_Data_MoreINFO.csv")
d2 = pd.read_csv("dedup_dallas.csv")
big = clean_yelp(dallas,d2)

In [5]:
big.to_csv("Cleaned_Dallas.csv")